## Neighborhoods in Toronto

Installing beautifull soup

In [143]:
!conda install -c anaconda beautifulsoup4 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



import libraries

In [144]:

import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
website_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup = BeautifulSoup(website_url, 'html.parser')


Extract table

In [145]:

table = soup.find('table', attrs={'class': "wikitable sortable"})

#column name
table_header=table.find_all('th')
table_header=[ele.text.strip() for ele in table_header]


In [146]:
#data table
table_body = table.find('tbody')
data=[]
rows = table_body.find_all('tr')
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    data.append([ele for ele in cols if ele]) # Get rid of empty valueta
dfn=pd.DataFrame(data=data)
dfn.columns=table_header


Preparing Data

In [148]:
#drop postal code null
dfn.dropna(0,inplace=True)
i=0
j=0
#change "Not assigned" in Borough and Neighborhood
for index, row in dfn.iterrows():
    
    if row["Borough"]=="Not assigned":
        #borrar regitro
        dfn.drop(index=[index],inplace=True)
        i=i+1
    else:
        if row["Neighbourhood"]=="Not assigned":
            #neighborhood will be the same as the borough
            val=dfn.iloc[index,1]
            dfn.at[index,"Neighbourhood"]=val
            j=j+i           


In [150]:
dfn.sort_values("Postcode")
df = pd.DataFrame(columns=table_header)
i=0
j=0
for i in range(dfn["Postcode"].count()):
    if i==0:
        pc=dfn.iloc[i,0]
        B=dfn.iloc[i,1]
        C=dfn.iloc[i,2]
    else:
        if dfn.iloc[i,0]==pc:
            C = C + ', '+ dfn.iloc[i,2]
        else:
            df = df.append({"Postcode":pc,"Borough":B,"Neighbourhood":C}, ignore_index=True)
            pc=dfn.iloc[i,0]
            B=dfn.iloc[i,1]
            C=dfn.iloc[i,2]
            j=j+1


In [151]:
df.shape

(102, 3)